In [ ]:
import os
import pandas as pd
from pyomo.environ import *
import glob

def solve_model(csv_path):
    # Reading the CSV file
    data = pd.read_csv(csv_path, header=None)

    # Define the model
    model = ConcreteModel()

    # Sets
    model.rows = RangeSet(0, 9)
    model.cols = RangeSet(0, 9)

    # Parameters
    model.water_requirement = Param(model.rows, model.cols, initialize=lambda model, i, j: data.iloc[i, j])

    # Decision variables
    model.water = Var(model.rows, model.cols, within=Binary)

    # Objective function (reformulate to be linear if possible)
    def obj_rule(model):
        total = sum(model.water[i, j] for i in model.rows for j in model.cols)
        return total

    model.obj = Objective(rule=obj_rule, sense=maximize)

    # Constraint function for available water
    def constraint_rule(model):
        return sum(model.water[i, j] * model.water_requirement[i, j] for i in model.rows for j in model.cols) <= 600

    model.constraint = Constraint(rule=constraint_rule)

    # Solve the model
    solver = SolverFactory('cbc')
    results = solver.solve(model)

    # Extract crop name from CSV filename
    crop_name = os.path.splitext(os.path.basename(csv_path))[0].split('_')[0]

    # Check if the solver successfully found a solution
    if results.solver.termination_condition == TerminationCondition.optimal:
        # Display results
        print("Crop:", crop_name)
        print("Objective value:", model.obj())
        print("Watering plan:")
        for i in model.rows:
            for j in model.cols:
                print(int(model.water[i, j].value), end=" ")
            print()

        # Extracting the water decision variable values
        water_values = [[int(model.water[i, j].value) for j in model.cols] for i in model.rows]

        # Counting the number of 1s
        num_ones = sum(sum(row) for row in water_values)

        # Displaying the number of 1s
        print("Number of fields to water:", num_ones)

        # Write results to file
        output_filename = f"{crop_name}_result.csv"
        with open(output_filename, 'w') as f:
            f.write("Crop: " + crop_name + "\n")
            f.write("Objective value: " + str(model.obj()) + "\n")
            f.write("Watering plan:\n")
            for row in water_values:
                f.write(' '.join(map(str, row)) + '\n')
            f.write("Number of fields to water: " + str(num_ones))

        return crop_name, model.obj()
    else:
        print("Solver failed to find an optimal solution for", csv_path)
        return crop_name, None

# Get all CSV files matching the pattern {crop_name}_water_diff.csv
csv_files = glob.glob("*_water_diff.csv")

# List to store objective values for each crop
objective_values = []

# Solve model for each CSV file
for csv_file in csv_files:
    crop_name, obj_value = solve_model(csv_file)
    if obj_value is not None:
        objective_values.append((crop_name, obj_value))

# Read filtered crops data
filtered_crops = pd.read_csv('/content/filtered_crops.csv')

# Update the objective value column
for crop_name, obj_value in objective_values:
    filtered_crops.loc[filtered_crops['crops'] == crop_name, 'objective_value'] = obj_value

# Save updated filtered crops data
filtered_crops.to_csv('/content/filtered_crops.csv', index=False)

# Display filtered crops with added objective values
print("Filtered Crops with Objective Values:")
print(filtered_crops)